## 공공데이터를 이용한 카페 상권분석(2023 Ver.)


**들어가며**

- 공공데이터를 통해 대한민국을 이해해봅시다(?).
- 공공데이터포털(data.go.kr)에 다양한 데이터가 공개되어 있습니다.
- 그 중에 카페(라는 업종분류)들에 대해서 현황을 조사하려고 합니다.

**명세사항**
1. 전국 카페 데이터를 모두 수집해야합니다.
2. 지역별 or 브랜드별 점포 현황을 확인합니다.
3. 분석 결과를 시각화합니다. 



[데이터] https://www.data.go.kr/data/15083033/fileData.do

In [1]:
# 라이브러리를 불러옵니다.
import os
import numpy as np
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import scipy as sp
import folium
import math
import itertools
import glob
import seaborn as sns
import unicodedata
import tqdm
import pyarrow

In [2]:
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

font_path = fm.findfont('AppleGothic')

if not font_path:
    print('Warning: AppleGothic font not found')
else:
    print("AppleGothic font found at ", font_path)

font_prop = fm.FontProperties(fname=font_path, size=12)

AppleGothic font found at  /System/Library/Fonts/Supplemental/AppleGothic.ttf


## 1. 데이터 불러오기

In [3]:
# 다운로드 받은 데이터중 일부를 열어봅니다.
# 소상공인시장진흥공단_상가(상권)정보_강원_202303

seoul = pd.read_csv("./data/소상공인시장진흥공단_상가(상권)정보_서울_202303.csv", usecols=list(range(2)), low_memory=True)
seoul

,상가업소번호,상호명
0,MA0101202210A0037544,영자's곱창
1,MA0101202210A0097916,호텔클로이
2,MA0101202210A0096343,나라차중계태권도장
3,MA0101202210A0093845,이상한스냅
4,MA010120220805430767,토끼정
...,...,...
474255,MA0101202302A0049414,행복한부동산중개사무소
474256,MA0101202302A0001316,좋은소식법률사무소
474257,MA0101202302A0036033,루젤라
474258,MA0101202302A0049679,153커피


In [4]:
file_paths

NameError: name 'file_paths' is not defined

In [5]:
# data 폴더에 있는 모든 csv 파일을 읽어오기 위해 glob을 사용합니다.


file_paths = sorted(glob.glob("./data/*.csv"))

total_df = pd.DataFrame()

for files in file_paths:
    temp = pd.read_csv(files, low_memory=True)
    total_df = pd.concat([total_df, temp])

total_df

# csv 목록 불러오기

# 모든 csv 병합하기


/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/3604496713.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(files, low_memory=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/3604496713.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(files, low_memory=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/3604496713.py:9: DtypeWarning: Columns (2,35) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(files, low_memory=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/3604496713.py:9: DtypeWarning: Columns (2,35) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(files, low_memory=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/3604496713.py:9: Dtyp

,상호명,지점명,시도명,시군구명,도로명주소,경도,위도,상가업소번호,상권업종대분류코드,상권업종대분류명,...,도로명,건물본번지,건물부번지,건물관리번호,건물명,구우편번호,신우편번호,동정보,층정보,호정보
0,공차,NaN,강원도,원주시,강원도 원주시 지정면 신지정로 211,127.872713,37.374189,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,한자루체험농원휴앤아크,NaN,강원도,인제군,강원도 인제군 기린면 방동약수로 68,128.388514,37.946792,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카페아지,NaN,강원도,속초시,강원도 속초시 설악산로 263-6,128.578466,38.163421,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,오슈,NaN,강원도,홍천군,강원도 홍천군 홍천읍 송학로3길 21,127.873714,37.684992,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,씨즐,NaN,강원도,강릉시,강원도 강릉시 경강로 2668,128.950185,37.770586,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83551,입시·교과학원,NaN,충청북도,청주시 흥덕구,충청북도 청주시 흥덕구 비하로45번길 21,127.423393,36.630861,MA0101202302A0095603,P1,교육,...,충청북도 청주시 흥덕구 비하로45번길,21.0,NaN,4311313800101380009008048,대궐빌라,361825.0,28366.0,NaN,NaN,NaN
83552,이마트24충북영동,센트럴점,충청북도,영동군,충청북도 영동군 영동읍 영동천2길 7,127.775502,36.173432,MA0101202302A0060289,G2,소매,...,충청북도 영동군 영동읍 영동천2길,7.0,NaN,4374025021106900004005126,NaN,370809.0,29145.0,NaN,NaN,NaN
83553,백운산장,NaN,충청북도,제천시,충청북도 제천시 수산면 상천1길 42,128.239583,36.967854,MA0101202302A0060299,I2,음식,...,충청북도 제천시 수산면 상천1길,42.0,NaN,4315033039102890000006119,NaN,390842.0,27218.0,NaN,NaN,NaN
83554,시투안강서위너스,NaN,충청북도,청주시 흥덕구,충청북도 청주시 흥덕구 비하로45번길 37,127.422521,36.630794,MA0101202302A0053211,G2,소매,...,충청북도 청주시 흥덕구 비하로45번길,37.0,NaN,4311313800101430000000001,NaN,361825.0,28366.0,NaN,2.0,NaN


In [6]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2557534 entries, 0 to 83555
Data columns (total 39 columns):
 #   Column     Dtype  
---  ------     -----  
 0   상호명        object 
 1   지점명        object 
 2   시도명        object 
 3   시군구명       object 
 4   도로명주소      object 
 5   경도         float64
 6   위도         float64
 7   상가업소번호     object 
 8   상권업종대분류코드  object 
 9   상권업종대분류명   object 
 10  상권업종중분류코드  object 
 11  상권업종중분류명   object 
 12  상권업종소분류코드  object 
 13  상권업종소분류명   object 
 14  표준산업분류코드   object 
 15  표준산업분류명    object 
 16  시도코드       float64
 17  시군구코드      float64
 18  행정동코드      float64
 19  행정동명       object 
 20  법정동코드      float64
 21  법정동명       object 
 22  지번코드       float64
 23  대지구분코드     float64
 24  대지구분명      object 
 25  지번본번지      float64
 26  지번부번지      float64
 27  지번주소       object 
 28  도로명코드      float64
 29  도로명        object 
 30  건물본번지      float64
 31  건물부번지      float64
 32  건물관리번호     object 
 33  건물명        object 
 34  구우편번호      float64
 

In [7]:
my_total_df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
my_total_df 

/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/2515537554.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  my_total_df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/2515537554.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  my_total_df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/2515537554.py:1: DtypeWarning: Columns (2,35) have mixed types. Specify dtype option on import or set low_memory=False.
  my_total_df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_59339/2515537554.py:1: DtypeWarning: Columns (2,35) have mixed types. Specify dtype option on import or set low_memory=False.

,상호명,지점명,시도명,시군구명,도로명주소,경도,위도,상가업소번호,상권업종대분류코드,상권업종대분류명,...,도로명,건물본번지,건물부번지,건물관리번호,건물명,구우편번호,신우편번호,동정보,층정보,호정보
0,공차,NaN,강원도,원주시,강원도 원주시 지정면 신지정로 211,127.872713,37.374189,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,한자루체험농원휴앤아크,NaN,강원도,인제군,강원도 인제군 기린면 방동약수로 68,128.388514,37.946792,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카페아지,NaN,강원도,속초시,강원도 속초시 설악산로 263-6,128.578466,38.163421,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,오슈,NaN,강원도,홍천군,강원도 홍천군 홍천읍 송학로3길 21,127.873714,37.684992,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,씨즐,NaN,강원도,강릉시,강원도 강릉시 경강로 2668,128.950185,37.770586,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557529,입시·교과학원,NaN,충청북도,청주시 흥덕구,충청북도 청주시 흥덕구 비하로45번길 21,127.423393,36.630861,MA0101202302A0095603,P1,교육,...,충청북도 청주시 흥덕구 비하로45번길,21.0,NaN,4311313800101380009008048,대궐빌라,361825.0,28366.0,NaN,NaN,NaN
2557530,이마트24충북영동,센트럴점,충청북도,영동군,충청북도 영동군 영동읍 영동천2길 7,127.775502,36.173432,MA0101202302A0060289,G2,소매,...,충청북도 영동군 영동읍 영동천2길,7.0,NaN,4374025021106900004005126,NaN,370809.0,29145.0,NaN,NaN,NaN
2557531,백운산장,NaN,충청북도,제천시,충청북도 제천시 수산면 상천1길 42,128.239583,36.967854,MA0101202302A0060299,I2,음식,...,충청북도 제천시 수산면 상천1길,42.0,NaN,4315033039102890000006119,NaN,390842.0,27218.0,NaN,NaN,NaN
2557532,시투안강서위너스,NaN,충청북도,청주시 흥덕구,충청북도 청주시 흥덕구 비하로45번길 37,127.422521,36.630794,MA0101202302A0053211,G2,소매,...,충청북도 청주시 흥덕구 비하로45번길,37.0,NaN,4311313800101430000000001,NaN,361825.0,28366.0,NaN,2.0,NaN


In [8]:
# 분석에 필요한 column을 고릅니다. ## 자유롭게 하셔도 상관없습니다.
total_df[~total_df["상호명"].isnull()]
temp[temp["상호명"].str.contains('커피')]

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
257,MA010120220805474761,꿀스커피,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4311310800112240000029013,퓨전빌딩,충청북도 청주시 서원구 월평로 7,362856,28790,NaN,NaN,NaN,127.485949,36.611330
354,MA010120220805492480,연두커피,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4311310400105550010000001,NaN,충청북도 청주시 흥덕구 사운로 292,361841,28467,NaN,NaN,NaN,127.471934,36.653481
426,MA010120220805508252,메가엠지씨커피청주오창,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4371025344006500001000789,JK영어전문학원,충청북도 청주시 청원구 오창읍 오창중앙로 123-7,363885,28123,NaN,NaN,NaN,127.416587,36.708989
549,MA010120220805525127,커피향에빠진사람들SARATOGA,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4374525032100940001004761,"증평인삼관광휴게소,증평인삼관광휴게소",충청북도 증평군 증평읍 중부로 2451,368906,27914,NaN,1,NaN,127.581759,36.795883
555,MA010120220805528543,커피방앗간,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4315010600102210001029682,NaN,충청북도 제천시 풍양로19길 15,390011,27173,NaN,NaN,NaN,128.212812,37.138115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83409,MA0101202302A0098142,김준호의대단한커피청주효촌점,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4371034022001880004067861,NaN,충청북도 청주시 상당구 남일면 단재로 500-4,360844,28187,NaN,1.0,NaN,127.508251,36.586703
83455,MA0101202302A0100129,약수터휴게소커피매점,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,4311112200100010003051652,NaN,충청북도 청주시 상당구 명암로293번길 8,360191,28311,NaN,1.0,NaN,127.527915,36.655212
83469,MA0101202302A0108935,레오커피,NaN,I2,음식,I212,비알코올,I21201,카페,I56111,...,4311112400101770011000003,NaN,충청북도 청주시 상당구 호미로 3,360101,28808,NaN,NaN,NaN,127.516364,36.613584
83498,MA0101202302A0060898,커피홀,충북영동점,I2,음식,I212,비알코올,I21201,카페,I56221,...,4374025021102050103000001,NaN,충청북도 영동군 영동읍 구교로 1,370802,29139,NaN,NaN,NaN,127.781658,36.174471


In [9]:
# 분석에 필요한 column을 고릅니다. ## 자유롭게 하셔도 상관없습니다.
total_df[total_df["상호명"].str.contains('커피', na=False)]


,상호명,지점명,시도명,시군구명,도로명주소,경도,위도,상가업소번호,상권업종대분류코드,상권업종대분류명,...,도로명,건물본번지,건물부번지,건물관리번호,건물명,구우편번호,신우편번호,동정보,층정보,호정보
5,피그놀리아커피,NaN,강원도,강릉시,강원도 강릉시 난설헌로 195,128.911401,37.790994,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,뉴커피,NaN,강원도,강릉시,강원도 강릉시 공항길127번길 22,128.950960,37.765453,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,구름커피,NaN,강원도,홍천군,강원도 홍천군 화촌면 설악로 2448,127.942873,37.732738,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,메가엠지씨커피동해천곡점,NaN,강원도,동해시,강원도 동해시 천곡로 14,129.106762,37.525141,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,메가엠지씨커피삼척,NaN,강원도,삼척시,강원도 삼척시 진주로 26,129.164350,37.443016,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83409,김준호의대단한커피청주효촌점,NaN,충청북도,청주시 상당구,충청북도 청주시 상당구 남일면 단재로 500-4,127.508251,36.586703,MA0101202302A0098142,I2,음식,...,충청북도 청주시 상당구 남일면 단재로,500.0,4.0,4371034022001880004067861,NaN,360844.0,28187.0,NaN,1.0,NaN
83455,약수터휴게소커피매점,NaN,충청북도,청주시 상당구,충청북도 청주시 상당구 명암로293번길 8,127.527915,36.655212,MA0101202302A0100129,I2,음식,...,충청북도 청주시 상당구 명암로293번길,8.0,NaN,4311112200100010003051652,NaN,360191.0,28311.0,NaN,1.0,NaN
83469,레오커피,NaN,충청북도,청주시 상당구,충청북도 청주시 상당구 호미로 3,127.516364,36.613584,MA0101202302A0108935,I2,음식,...,충청북도 청주시 상당구 호미로,3.0,NaN,4311112400101770011000003,NaN,360101.0,28808.0,NaN,NaN,NaN
83498,커피홀,충북영동점,충청북도,영동군,충청북도 영동군 영동읍 구교로 1,127.781658,36.174471,MA0101202302A0060898,I2,음식,...,충청북도 영동군 영동읍 구교로,1.0,NaN,4374025021102050103000001,NaN,370802.0,29139.0,NaN,NaN,NaN


In [ ]:
# for i, word in enumerate(total_df['상권업종소분류명'].unique()):
#     print(i, word)

df_cafe = total_df[total_df["상권업종소분류명"] == "카페"]
print(df_cafe.info(memory_usage="deep"))
df_cafe

In [ ]:
# 메모리 낭비를 막기 위해 필요없는 변수는 제거합니다.
import gc

del total_df
gc.collect()

## 2. 데이터 구경하기

#### 전국 커피 전문점 

In [ ]:
df_cafe[["시도명","시군구명","도로명주소","경도","위도"]]


In [ ]:
df_cafe[["시도명"]]

In [ ]:
print(df_cafe["시도명"].unique())
print("*"*8)
print(df_cafe["시도명"].value_counts())
print("*"*8)
print(df_cafe["시도명"].nunique())

In [ ]:
usecols = ["상호명","지점명","시도명","시군구명","도로명주소","경도","위도"]
df_cafe = df_cafe[usecols]

In [ ]:
df_cafe.to_excel("../Public_Data_PJT/data/cafes_230503.xlsx", index = False)

In [ ]:
df_cafe.to_csv("../Public_Data_PJT/data/cafes_230503.csv", index = False)

In [ ]:
!pip install pyarrow
import pyarrow.parquet

df_cafe.to_parquet("../Public_Data_PJT/data/cafes_230503.parquet", index = False)

In [ ]:
import pyarrow.feather
df_cafe.to_feather("../Public_Data_PJT/data/cafes_230503.feather")

#### 서울내 커피 전문점 

In [ ]:
# 카페 중에 "서울"에 위치하고 있는 점포만 뽑아냅니다.
cafe_seoul = df_cafe["시도명"].str.contains("서울") 
cafe_seoul

In [ ]:
print ("%.3f%%" % (len(cafe_seoul) / len(df_cafe) * 100))
print (f"{len(cafe_seoul) / len(df_cafe) * 100:.31}%")
print ("{:.3f}%". format(len(cafe_seoul) / len(df_cafe) * 100))

#### 전국 스타벅스가 없어져서 '컴포즈'

In [ ]:
# 이번엔 전국에 있는 컴포즈커피를 뽑아냅니다.
df_cafe[df_cafe["상호명"].str.contains("이디야")]
df_cafe[df_cafe["상호명"].str.contains("컴포즈")]
df_cafe[df_cafe["상호명"].str.contains("[메가][엠지씨|MGC|mgc]*[커피]")]
df_cafe[df_cafe["상호명"].str.contains("빽다방")]

cond1 = df_cafe["상호명"].str.contains("컴포즈|컴포스")
cond2 = df_cafe["상호명"] == "컴퍼스"
df_compose = df_cafe[cond1 | cond2]
df_compose_seoul = df_compose[df_compose["시도명"]== "서울특별시"]
print(df_compose_seoul)
print(df_compose.shape, df_compose_seoul.shape)

In [ ]:
# 서울에 있는 비율은?

print("전체 컴포즈 직영점 중 서울에 있는 비율은: %.3f%%"% ((len(df_compose_seoul)/len(df_compose))*100))

#### 서울 스타벅스

In [ ]:
# 이번엔 서울에 있는 스타벅스를 뽑아냅니다.
df_seoul_starbucks = 


#### 전국 이디야

In [ ]:
# 이번엔 전국에 있는 이디야를 뽑아냅니다.


#### 서울 이디야

In [ ]:
# 이번엔 서울에 있는 스타벅스를 뽑아냅니다.


하다보니, 몹시 귀찮아졌습니다..! 특정 브랜드의 커피브랜드 정보를 추출하는 함수를 만듭니다.

In [ ]:
def get_cafe_info():
    
    return

또, 어떤 브랜드들을 추가로 계속 살펴봐야할까요?

Source : 

- https://franchise.ftc.go.kr/mnu/00014/program/firHope/view.do
- https://brikorea.com/bbs/board.php?bo_table=rep_1&wr_id=2363

#### 전국 컴포즈 

In [ ]:
df_compose = 

#### 서울 컴포즈

In [ ]:
df_seoul_compose = 

#### 전국 투썸 

#### 서울 투썸 

#### 전국 빽다방 

#### 서울 빽다방 

#### 전국 할리스 

#### 서울 할리스 

#### 전국 메가커피

#### 서울 메가커피 

## 3. 커피전문점 별 비율 비교하기 (주요 브랜드 위주로)

**2023년 4월 기준 커피전문점 평판 순위**

- 커피브랜드 평판 순위 찾기

1. 스타벅스
2. 메가커피
3. 투썸
4. 이디야
5. 컴포즈

**변수**

- 전체 점포 : data
- 전체/서울 커피전문점 : df_coffee / df_seoul_starbucks



- 전체/서울 스타벅스 : df_starbucks / df_seoul_starbucks
- 전체/서울 이디야 : df_ediya / df_seoul_ediya
- 전체/서울 커피빈 : df_coffeebean / df_seoul_coffeebean
- 전체/서울 투썸플레이스 : df_twosome / df_seoul_twosome
- 전체/서울 빽다방 : df_bbaek / df_seoul_bbaek
- 전체/서울 할리스 : df_hollys / df_seoul_hollys
- 전체/서울 메가커피 : df_mega / df_seoul_mega
- 전체/서울 컴포즈 : df_compose / df_seoul_compose

### 1) 전체 커피전문점 내 주요 커피브랜드 입점 비율 

In [ ]:
n_cafe = len(df_cafe)
n_starbucks = len(df_starbucks)
n_seoul_starbucks = len(df_seoul_starbucks)
n_twosome = len(df_twosome)
n_seoul_twosome = len(df_seoul_twosome)
n_ediya = len(df_ediya)
n_seoul_ediya = len(df_seoul_ediya)
n_coffeebean = len(df_coffeebean)
n_seoul_coffeebean = len(df_seoul_coffeebean)
...

### 2) 서울 커피전문점 내 주요 커피브랜드 입점 비율 

### 3) 각 커피브랜드별 서울 입점 비율 

In [ ]:
# 각 구별로 스타벅스가 얼마나 있는지 확인합니다.

In [ ]:
# 시각화를 위한 라이브러리를 불러옵니다.
# 한글 글꼴을 추가해볼까요?


In [ ]:
# 주요 5대 커피브랜드 서울 입점 비율을 시각화합니다.


### 4. GIS를 이용하여 스타벅스 상권 분석을 해봅시다.

- 업계에 알려져 있는 속설(?)이 있습니다. 스타벅스 근처에 이디야 카페가 들어선다. 정말 진짜인지 확인해봅시다.


- 근처의 기준을 정해야합니다.

- 지도 상에서 시각화를 해야합니다.


(이런 귀찮은 코드를 구현해야할때, 어떻게 할 수 있는지 찾아봅시다)